# Ex2 - Filtering and Sorting Data

This time we are going to pull data directly from the internet.

### Step 1. Import the necessary libraries

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MyApp').getOrCreate()

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/02_Filtering_%26_Sorting/Euro12/Euro_2012_stats_TEAM.csv). 

### Step 3. Assign it to a variable called euro12.

In [2]:
euro12 = spark.read.csv('Euro_2012_stats_TEAM.csv', header=True, inferSchema=True)

### Step 4. Select only the Goal column.

In [3]:
euro12[['Goals']].show()

+-----+
|Goals|
+-----+
|    4|
|    4|
|    4|
|    5|
|    3|
|   10|
|    5|
|    6|
|    2|
|    2|
|    6|
|    1|
|    5|
|   12|
|    5|
|    2|
+-----+



### Step 5. How many team participated in the Euro2012?

In [4]:
euro12[['Team']].distinct().count()

16

### Step 6. What is the number of columns in the dataset?

In [5]:
len(euro12.columns)

35

### Step 7. View only the columns Team, Yellow Cards and Red Cards and assign them to a dataframe called discipline

In [6]:
discipline = euro12[['Team','Yellow Cards','Red Cards']]
discipline.show()

+-------------------+------------+---------+
|               Team|Yellow Cards|Red Cards|
+-------------------+------------+---------+
|            Croatia|           9|        0|
|     Czech Republic|           7|        0|
|            Denmark|           4|        0|
|            England|           5|        0|
|             France|           6|        0|
|            Germany|           4|        0|
|             Greece|           9|        1|
|              Italy|          16|        0|
|        Netherlands|           5|        0|
|             Poland|           7|        1|
|           Portugal|          12|        0|
|Republic of Ireland|           6|        1|
|             Russia|           6|        0|
|              Spain|          11|        0|
|             Sweden|           7|        0|
|            Ukraine|           5|        0|
+-------------------+------------+---------+



### Step 8. Sort the teams by Red Cards, then to Yellow Cards

In [7]:
from pyspark.sql.functions import col

In [8]:
discipline.orderBy(col('Red Cards').desc(), col('Yellow Cards').desc()).show(16)

+-------------------+------------+---------+
|               Team|Yellow Cards|Red Cards|
+-------------------+------------+---------+
|             Greece|           9|        1|
|             Poland|           7|        1|
|Republic of Ireland|           6|        1|
|              Italy|          16|        0|
|           Portugal|          12|        0|
|              Spain|          11|        0|
|            Croatia|           9|        0|
|     Czech Republic|           7|        0|
|             Sweden|           7|        0|
|             France|           6|        0|
|             Russia|           6|        0|
|            England|           5|        0|
|        Netherlands|           5|        0|
|            Ukraine|           5|        0|
|            Denmark|           4|        0|
|            Germany|           4|        0|
+-------------------+------------+---------+



### Step 9. Calculate the mean Yellow Cards given per Team

In [9]:
discipline[['Yellow Cards']].groupBy().mean().show()

+-----------------+
|avg(Yellow Cards)|
+-----------------+
|           7.4375|
+-----------------+



### Step 10. Filter teams that scored more than 6 goals

In [10]:
euro12[['Team', 'Goals']].where(col('Goals')>6).show()

+-------+-----+
|   Team|Goals|
+-------+-----+
|Germany|   10|
|  Spain|   12|
+-------+-----+



### Step 11. Select the teams that start with G

In [11]:
from pyspark.sql.functions import regexp_extract, substring

In [12]:
euro12[['Team']].where(substring(col('Team'), 0,1) == 'G' ).show()

+-------+
|   Team|
+-------+
|Germany|
| Greece|
+-------+



### Step 12. Select the first 7 columns

In [13]:
col7 = [col if '.' not in col else '`'+col+'`' for col in euro12.columns[:7]]

In [14]:
euro12[col7].show()

+-------------------+-----+---------------+----------------+-----------------+----------------+--------------------------+
|               Team|Goals|Shots on target|Shots off target|Shooting Accuracy|% Goals-to-shots|Total shots (inc. Blocked)|
+-------------------+-----+---------------+----------------+-----------------+----------------+--------------------------+
|            Croatia|    4|             13|              12|            51.9%|           16.0%|                        32|
|     Czech Republic|    4|             13|              18|            41.9%|           12.9%|                        39|
|            Denmark|    4|             10|              10|            50.0%|           20.0%|                        27|
|            England|    5|             11|              18|            50.0%|           17.2%|                        40|
|             France|    3|             22|              24|            37.9%|            6.5%|                        65|
|            Ger

### Step 13. Select all columns except the last 3.

In [15]:
col_expect_last3 = [col if '.' not in col else '`'+col+'`' for col in euro12.columns[:-3]]

In [16]:
euro12[col_expect_last3].show()

+-------------------+-----+---------------+----------------+-----------------+----------------+--------------------------+------------+-------------+--------------------+------------+------+----------------+----------------+-------+-------+--------+-------------+-------+----------+-------------+-------------------+------------+------+--------------+----------+--------------------+---------+--------------+--------+------------+---------+
|               Team|Goals|Shots on target|Shots off target|Shooting Accuracy|% Goals-to-shots|Total shots (inc. Blocked)|Hit Woodwork|Penalty goals|Penalties not scored|Headed goals|Passes|Passes completed|Passing Accuracy|Touches|Crosses|Dribbles|Corners Taken|Tackles|Clearances|Interceptions|Clearances off line|Clean Sheets|Blocks|Goals conceded|Saves made|Saves-to-shots ratio|Fouls Won|Fouls Conceded|Offsides|Yellow Cards|Red Cards|
+-------------------+-----+---------------+----------------+-----------------+----------------+-----------------------

### Step 14. Present only the Shooting Accuracy from England, Italy and Russia

In [17]:
conditions = [col('Team') == country for country in ['England', 'Italy', 'Russia']]

In [18]:
conditions

[Column<'(Team = England)'>,
 Column<'(Team = Italy)'>,
 Column<'(Team = Russia)'>]

In [20]:
euro12[['Shooting Accuracy', 'Team']].where(conditions[0] | conditions[1] | conditions[2]).show()

+-----------------+-------+
|Shooting Accuracy|   Team|
+-----------------+-------+
|            50.0%|England|
|            43.0%|  Italy|
|            22.5%| Russia|
+-----------------+-------+



In [21]:
euro12[['Team','Shooting Accuracy']].where(col('Team').isin(['England','Italy','Russia'])).show()

+-------+-----------------+
|   Team|Shooting Accuracy|
+-------+-----------------+
|England|            50.0%|
|  Italy|            43.0%|
| Russia|            22.5%|
+-------+-----------------+

